In [1]:
!pip3 install numpy --upgrade
!pip3 install scipy --upgrade

In [2]:
import fft_project
import psutil
import logging
import time
import scipy
import numpy as np
from importlib import reload
from fft_project import base


In [3]:
'''Below are the testcases, where we controlled the multithreading and creating copies
of our version of 2DFFT versus Scipy fft2'''

#Printing out Current CPU Info
print("="*40, "CPU Info", "="*40)
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")

======================================== CPU Info ========================================
Physical cores: 4
Total cores: 8
Max Frequency: 2800.00Mhz
Min Frequency: 2800.00Mhz
Current Frequency: 2800.00Mhz


In [4]:
def random_matrix(n, m):
    logging.info(f'Matrix shape: {n}x{m}')
    result = np.zeros((n, m), dtype=np.complex128)
    result.real = np.random.rand(n, m)
    return result

reload(logging)
base.prepare_logger()


Measurements with multithreading disabled, and no type conversions

In [5]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    start = time.time()
    expected = scipy.fft.fft2(result)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    start = time.time()
    result = fft_project.fft2d(result, return_copy=False, use_threads=False)
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    expected = abs(expected)
    result = abs(result)
    correctness = np.allclose(result, expected, atol=0.05)
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=False, use_threads=False)
    start = time.time()
    inversed_scipy = scipy.fft.ifft2(transformed)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    inversed_our = transformed
    start = time.time()
    inversed_our = fft_project.fft2d(
        inversed_our, return_copy=False, use_threads=False, inverse=True
    )
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    inversed_scipy = abs(inversed_scipy)
    inversed_our = abs(inversed_our)
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    assert correctness

fft_cpp_impl_test()
inverse_test()

[INFO]  Matrix shape: 4096x8192
[INFO]  Scipy: 2.1821608543395996
[INFO]  Our: 1.1811769008636475
[INFO]  54.13% of Scipy time!
[INFO]  Matrix shape: 8192x4096
[INFO]  Scipy: 1.7597148418426514
[INFO]  Our: 1.2059638500213623
[INFO]  68.53% of Scipy time!


With multithreading enabled, no type conversions

In [6]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    start = time.time()
    expected = scipy.fft.fft2(result, workers = -1)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    start = time.time()
    result = fft_project.fft2d(result, return_copy=False, use_threads=True)
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    expected = abs(expected)
    result = abs(result)
    correctness = np.allclose(result, expected, atol=0.05)
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=False, use_threads=True)
    start = time.time()
    inversed_scipy = scipy.fft.ifft2(transformed, workers = -1)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    inversed_our = transformed
    start = time.time()
    inversed_our = fft_project.fft2d(
        inversed_our, return_copy=False, use_threads=True, inverse=True
    )
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    inversed_scipy = abs(inversed_scipy)
    inversed_our = abs(inversed_our)
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    assert correctness

fft_cpp_impl_test()
inverse_test()

[INFO]  Matrix shape: 4096x8192
[INFO]  Scipy: 0.5782830715179443
[INFO]  Our: 0.5804431438446045
[INFO]  100.37% of Scipy time!
[INFO]  Matrix shape: 8192x4096
[INFO]  Scipy: 0.6545920372009277
[INFO]  Our: 0.6854162216186523
[INFO]  104.71% of Scipy time!


No multithreading, with type conversions

In [7]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    start = time.time()
    expected = scipy.fft.fft2(result)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    start = time.time()
    result = fft_project.fft2d(result, return_copy=True, use_threads=False)
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    expected = abs(expected)
    result = abs(result)
    correctness = np.allclose(result, expected, atol=0.05)
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=True, use_threads=False)
    start = time.time()
    inversed_scipy = scipy.fft.ifft2(transformed)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    inversed_our = transformed
    start = time.time()
    inversed_our = fft_project.fft2d(
        inversed_our, return_copy=True, use_threads=False, inverse=True
    )
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    inversed_scipy = abs(inversed_scipy)
    inversed_our = abs(inversed_our)
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    assert correctness

fft_cpp_impl_test()
inverse_test()

[INFO]  Matrix shape: 4096x8192


[INFO]  Scipy: 1.7507572174072266
[INFO]  Our: 1.6739509105682373
[INFO]  95.61% of Scipy time!
[INFO]  Matrix shape: 8192x4096
[INFO]  Scipy: 1.725538969039917
[INFO]  Our: 1.7178521156311035
[INFO]  99.55% of Scipy time!


No multithreading, no type conversions

In [8]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    start = time.time()
    expected = scipy.fft.fft2(result)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    start = time.time()
    result = fft_project.fft2d(result, return_copy=False, use_threads=False)
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    expected = abs(expected)
    result = abs(result)
    correctness = np.allclose(result, expected, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=False, use_threads=False)
    start = time.time()
    inversed_scipy = scipy.fft.ifft2(transformed)
    scipy_time = time.time() - start
    logging.info(f'Scipy: {scipy_time}')
    inversed_our = transformed
    start = time.time()
    inversed_our = fft_project.fft2d(
        inversed_our, return_copy=True, use_threads=False, inverse=True
    )
    our_time = time.time() - start
    logging.info(f'Our: {our_time}')
    logging.info(f'{our_time / scipy_time * 100:.2f}% of Scipy time!')
    inversed_scipy = abs(inversed_scipy)
    inversed_our = abs(inversed_our)
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

fft_cpp_impl_test()
inverse_test()


[INFO]  Matrix shape: 4096x8192
[INFO]  Scipy: 1.7898519039154053
[INFO]  Our: 1.0755929946899414
[INFO]  60.09% of Scipy time!
[INFO]  Is the result correct: True
[INFO]  Matrix shape: 8192x4096
[INFO]  Scipy: 1.75516676902771
[INFO]  Our: 1.4335732460021973
[INFO]  81.68% of Scipy time!
[INFO]  Is the result correct: True
